Linear kernel과 RBF kernel은 왜 서로 다른 비율로 support vector를 선택하게 되는가? (왜 linear kernel만 sparse support vector를 지니는가?) 

좀 더 자세하게 linear kernel과 rbf kernel을 썼을 때 각각 어떤 classification boundary를 따는 걸까? 

l30_r15 데이터로만 한 번 살펴보자

In [1]:
from sklearn.svm import SVC
from sklearn.preprocessing import normalize
from py.utils import load_data
import pickle

import warnings
warnings.filterwarnings('ignore')

directory = '../data/'
head = 'l30_r15'

x, y, x_words, vocabs = load_data(head, directory)
x = normalize(x)

x shape = (15105, 2770)
y shape = (15105,)
# features = 2770
# L words = 15105


In [2]:
svm_linear = SVC(C=1.0, kernel='linear')
svm_linear.fit(x, y)
print('%s support vector %.3f' % ('%', svm_linear.n_support_.sum()/x.shape[0]))

% support vector 0.027


In [3]:
svm_rbf = SVC(C=1.0, kernel='rbf')
svm_rbf.fit(x, y)
print('%s support vector %.3f' % ('%', svm_rbf.n_support_.sum()/x.shape[0]))

% support vector 0.332


## Linear kernel을 알아보자

In [4]:
neg = svm_linear.support_[:svm_linear.n_support_[0]].tolist()
pos = svm_linear.support_[svm_linear.n_support_[0]:].tolist()

neg_words = [x_words[i] for i in neg]
pos_words = [x_words[i] for i in pos]

In [5]:
pos_words

['나머지',
 '영배',
 '결국',
 '생동',
 '어림',
 '성익',
 '새롬이',
 '것쯤',
 '미동',
 '거기',
 '걔네',
 '번쯤',
 '불문',
 '이것',
 '경화',
 '저',
 '국내적',
 '얘긴',
 '재직',
 '뚱쇠',
 '얘들',
 '태성',
 '일반이론',
 '창권',
 '눈곱',
 '뒷짐',
 '재영이',
 '총력',
 '여기저기',
 '훈',
 '마중',
 '멱살',
 '참상',
 '넷째',
 '위용',
 '예사',
 '윤재홍',
 '다비',
 '저명',
 '한창',
 '상무님',
 '발버둥',
 '장하일',
 '예리',
 '장군이',
 '형언',
 '법석',
 '위배',
 '기겁',
 '개중',
 '한은',
 '웬일',
 '드러커',
 '똘배',
 '둘째',
 '단조',
 '혼자',
 '동욱',
 '요새',
 '세진',
 '미경',
 '안중',
 '성태',
 '언제',
 '팔짱',
 '한때',
 '원산지',
 '가르시아',
 '장군님',
 '안수',
 '시기적',
 '발원',
 '제록스',
 '반쯤',
 '관섭',
 '때쯤',
 '진아',
 '구속기소',
 '권익',
 '저거',
 '뿐',
 '아랑곳',
 '즉석',
 '후보쪽',
 '자초지종',
 '혹자',
 '첫째',
 '개의',
 '품이',
 '퉁명',
 '이거',
 '막돌초석',
 '업종별',
 '소식통',
 '허허씨',
 '좆',
 '소식통들',
 '물론',
 '나',
 '지윤',
 '경찰청',
 '갑동이',
 '당국자',
 '언론들',
 '둘이',
 '상현',
 '무어',
 '셋째',
 '순덕',
 '윤수',
 '허울',
 '인제',
 '재판부',
 '그니',
 '가운데',
 '테',
 '다섯째',
 '통용',
 '경혜',
 '복지부',
 '거리낌',
 '기상청',
 '십',
 '한번',
 '일쑤',
 '두혁',
 '동준',
 '정말',
 '도리질',
 '외무장관',
 '상덕',
 '반색',
 '여경',
 '강아지똥',
 '한동안',
 '명이',
 '명심',